<a href="https://colab.research.google.com/github/aishwaryar12309/gaussian-blur-stanford-dogs/blob/main/GaussianBlurStanfordDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
import tensorflow_datasets as tfds
from tensorflow.keras import datasets, layers, models, Input
from keras import Model
from tensorflow.python.distribute.cross_device_ops import validate_destinations

#Stanford Dogs Dataset
- Used for ImageNet - http://vision.stanford.edu/aditya86/ImageNetDogs/
- Annotations of class and bounding boxes
- 12 classes (dog breeds)


In [ ]:
dataset, metadata = tfds.load('stanford_dogs', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [3]:
metadata.features['label'].names[0]

'n02085620-chihuahua'

In [6]:
num_train_examples = metadata.splits['train'].num_examples #1200
num_test_examples = metadata.splits['test'].num_examples #8580
class_names = metadata.features['label'].names #breednames


# Resize Train Dataset and Create Batches
- (224, 224, 3) expected input shape
- 255: 8 bits, 2^8 = 256 - 1 = 255 is the highest value
- batch size : choose some power of 2

In [7]:
image_size = (224, 224)
batch_size = 64
def normalize(img, label):
  norm_image = tf.image.resize(img, image_size)
  norm_image /= 255
  return norm_image, label

train_dataset=train_dataset.map(normalize)
test_dataset=test_dataset.map(normalize)
training_batches = train_dataset.cache().shuffle(num_train_examples//4).batch(batch_size).prefetch(1)
testing_batches = test_dataset.cache().shuffle(num_train_examples//4).batch(batch_size).prefetch(1)

#Keras Layers for Preprocessing and Augmentation
- RandomFlip, RandomZoom, RandomRotation

In [8]:
rescale = tf.keras.Sequential([layers.Rescaling(1./255, input_shape=(224, 224, 3))])
flip_zoom = tf.keras.Sequential([layers.RandomFlip(mode='HORIZONTAL_AND_VERTICAL'), layers.RandomZoom(0.2)])

## Apply Gaussian Blur to Dataset Before Training

In [ ]:
def fspecial_gauss(size, sigma=0.5):
    x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()

def gauss2D(shape=(3,3),sigma=0.5):

    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

In [ ]:
kernel_size = 3
kernel_weights = gauss2D(shape=(kernel_size,kernel_size))
print(kernel_weights)

[[0.01134374 0.08381951 0.01134374]
 [0.08381951 0.61934703 0.08381951]
 [0.01134374 0.08381951 0.01134374]]


In [ ]:
channels = 3
kernel_weights = np.expand_dims(kernel_weights, axis=-1)
kernel_weights = np.repeat(kernel_weights, channels, axis=-1) # apply the same filter on all the input channels
kernel_weights = np.expand_dims(kernel_weights, axis=-1)
gauss = tf.keras.layers.DepthwiseConv2D(kernel_size, use_bias = False, padding='same')

ValueError: Unrecognized keyword arguments passed to Model: {'input': <KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor_38>, 'output': <KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor_39>}

In [9]:
model = models.Sequential([
    rescale,
    flip_zoom,
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape = (224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(600, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation=tf.nn.softmax)
    ])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.compile(optimizer= "adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

history = model.fit(
    training_batches,
    epochs = 100,
    steps_per_epoch = 120
)


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


120/120 ━━━━━━━━━━━━━━━━━━━━ 29s 162ms/step - accuracy: 0.0074 - loss: 4.7884
Epoch 2/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 11s 93ms/step - accuracy: 0.0063 - loss: 4.7886
Epoch 3/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 111ms/step - accuracy: 0.0073 - loss: 4.7876
Epoch 4/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.0039 - loss: 4.7885 
Epoch 5/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 111ms/step - accuracy: 0.0092 - loss: 4.7877
Epoch 6/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.0027 - loss: 4.7883 
Epoch 7/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 110ms/step - accuracy: 0.0060 - loss: 4.7877
Epoch 8/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.0052 - loss: 4.7884 
Epoch 9/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 111ms/step - accuracy: 0.0075 - loss: 4.7878
Epoch 10/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.0067 - loss: 4.7884 
Epoch 11/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 111ms/step - accuracy: 0.0068 - loss: 4.7878
Epoch 12/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.0052 - loss: 4.7883 
Epoch 13/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 111ms/step - accuracy: 0.0065 - loss: 4.7878
Epoch 14/100
120/120 ━━

In [ ]:
results = model.evaluate(testing_batches, batch_size=64)